# Voting Regressor

In [1]:
from sklearn.datasets import fetch_california_housing 
housing = fetch_california_housing(as_frame=True)

In [2]:
housing.data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [3]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [4]:
housing.target_names

['MedHouseVal']

In [5]:
housing.data.shape

(20640, 8)

In [6]:
X = housing.data
y = housing.target

In [7]:
# test-train-split
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y ,test_size=0.2,random_state = 42)

In [8]:
#scale the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Creating the individual regression model 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor , VotingRegressor
from xgboost import XGBRegressor
lin_reg = LinearRegression()
svg_reg = SVR(kernel = 'rbf')
xgb_reg = XGBRegressor(random_state=42)
rf_reg = RandomForestRegressor(random_state=42)


In [10]:
# Creating a voting Regressor with the individual model
voting_reg = VotingRegressor(estimators = [('lr',lin_reg),('svr',svg_reg),('xgb',xgb_reg),('rf',rf_reg)],weights=[0.5, 0.8, 1.5, 1.2],  # More weight to stronger models
    n_jobs=-1)
#fitting the Regressor model on the training data
voting_reg.fit(X_train_scaled,y_train)

VotingRegressor(estimators=[('lr', LinearRegression()), ('svr', SVR()),
                            ('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          feature_weights=None, gamma=None,
                                          grow_policy=None,
                                          im...
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None, ...)),
                            ('rf', RandomForestRegressor(random_state=42))],
                n_jobs=-1, weights=[0.5, 0.8, 1.5, 1.2])

In [11]:
#prediction 
y_pred =voting_reg.predict(X_test_scaled)

In [12]:
from sklearn.metrics import r2_score , mean_squared_error
import numpy as np
# Step 9: Evaluate performance
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Voting Regressor Performance:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

Voting Regressor Performance:
R² Score: 0.8156
RMSE: 0.4915


In [13]:
# Optional: Compare with individual models
for name, model in [('Linear Regression', lin_reg), ('Random Forest', rf_reg), ('Gradient Boosting', xgb_reg),('SVM',svg_reg)]:
    model.fit(X_train_scaled, y_train)
    pred = model.predict(X_test_scaled)
    print(f"{name}: R² = {r2_score(y_test, pred):.4f}, RMSE = {np.sqrt(mean_squared_error(y_test, pred)):.4f}")


Linear Regression: R² = 0.5758, RMSE = 0.7456
Random Forest: R² = 0.8053, RMSE = 0.5051
Gradient Boosting: R² = 0.8301, RMSE = 0.4718
SVM: R² = 0.7276, RMSE = 0.5975


# Bagging Regressor

In [19]:
from sklearn.ensemble import BaggingRegressor
# Define the base estimator
base_estimator = BaggingRegressor(random_state=42)
#intialize the bagging Regressor
bag_reg = BaggingRegressor(
    estimator = base_estimator,
    n_estimators = 50, #number of Trees
    max_samples = 1.0, #Use all the samples per base estimator
    max_features = 1.0, # Use all features per base estimator
    bootstrap = True,
    n_jobs = -1 #uses all the CPU cores for speed
)
# Train the Bagging Regressor
bag_reg.fit(X_train,y_train)
#Mkae prediction
y_pred = bag_reg.predict(X_test)
r2 = r2_score(y_test,y_pred)
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print("Bagging Regressor Results on california housing dataset")
print(f"R2 square :{r2:.4f}")
print(f"RMSE : {rmse:.4f}")

Bagging Regressor Results on california housing dataset
R2 square :0.8005
RMSE : 0.5113


# Boosting Regressor

In [21]:
# 📘 Import libraries
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import numpy as np
import warnings
# warnings.filterwarnings('ignore')

# # 1️⃣ Load dataset
# data = fetch_california_housing()
# X, y = data.data, data.target

# # 2️⃣ Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3️⃣ Initialize boosting regressors
models = {
    "AdaBoost" : AdaBoostRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    "GradientBoosting" : GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    "XGBoost" : XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    "LightGBM" : LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

# 4️⃣ Train & evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"\n🔹 {name} Regressor 🔹")
    print(f"R² Score: {r2:.4f}")
    print(f"RMSE: {rmse:.4f}")



🔹 AdaBoost Regressor 🔹
R² Score: 0.5664
RMSE: 0.7538

🔹 GradientBoosting Regressor 🔹
R² Score: 0.7756
RMSE: 0.5422

🔹 XGBoost Regressor 🔹
R² Score: 0.7747
RMSE: 0.5433
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000299 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1838
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.071947

🔹 LightGBM Regressor 🔹
R² Score: 0.8360
RMSE: 0.4635
